# [example](https://suhas.org/function-call-ast-python/)

* https://stackoverflow.com/questions/1515357/simple-example-of-how-to-use-ast-nodevisitor
* https://suhas.org/function-call-ast-python/
* https://docs.python.org/3/library/ast.html#ast.NodeVisitor

In [1]:
import ast
import astor  # https://stackoverflow.com/questions/36022935/rewriting-code-with-ast-python
# Python itself doesn’t provide a way to turn a compiled code object into an AST, or an AST into a string of code. https://greentreesnakes.readthedocs.io/en/latest/tofrom.html

class FunctionCallVisitor(ast.NodeVisitor):
    """
    we will use this to find all callables ("Call" nodes in an ast). then, once we
    are on a Call node, we print its information. a Call node could be defined by 
    FunctionDef but also ImportFrom.
    
    if we wanted to visit other types of nodes in the ast, we would replace visit_Call
    with visit_Assign, visit_Import, etc...
    
    https://stackoverflow.com/questions/1515357/simple-example-of-how-to-use-ast-nodevisitor
    """
    def visit_Call(self, node):
        # print(ast.dump(node))
        print(node.func.id)
        
# # using FunctionCallVisitor, we can find its subroutines... which is func A... 
# lets fetch the subroutines name, we cant return values so we capture stdout
# https://stackoverflow.com/questions/16571150/how-to-capture-stdout-output-from-a-python-function-call
import io
from contextlib import redirect_stdout

def find_callables(curr):
    """for a given node, find its callables.
    """
    f = io.StringIO()
    with redirect_stdout(f):
        FunctionCallVisitor().visit(curr)
    out = f.getvalue()
    return out.split()


# return the index of tree.body that corresponds to the name
def name2idx(target):
    """
    returns the index, an int, of the tree body that corresponds
    to the target, a string.
    """
    for i, node in enumerate(tree.body):        
        # we only want funcdef statements
        if isinstance(node, ast.FunctionDef):
            if node.name == target:
                idx = i
                return idx
    return False

In [2]:
# load source into memory
with open('hello.py') as f: 
    source = f.read()

In [3]:
# convert source code into ast, which is a tree
tree = ast.parse(source)
tree.body

In [4]:
# this is a flattened view of the ast - this is only for illustrative purposes and we wont use this
for statement in tree.body:
    print(ast.dump(statement), '\n')

Import(names=[alias(name='ast', asname=None)]) 

FunctionDef(name='a', args=arguments(args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Assign(targets=[Name(id='x', ctx=Store())], value=Num(n=2)), Return(value=BinOp(left=BinOp(left=Call(func=Name(id='c', ctx=Load()), args=[], keywords=[]), op=Add(), right=Call(func=Name(id='b', ctx=Load()), args=[], keywords=[])), op=Add(), right=Name(id='x', ctx=Load())))], decorator_list=[], returns=None) 

FunctionDef(name='b', args=arguments(args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=Num(n=2), op=Add(), right=Call(func=Name(id='c', ctx=Load()), args=[], keywords=[])))], decorator_list=[], returns=None) 

FunctionDef(name='c', args=arguments(args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=Num(n=3))], decorator_list=[], returns=None) 

FunctionDef(name='d', args=arguments(args=[], vararg=None, kwonlyar

In [5]:
"""
we cant build subset using this approach because what we really care about is building a call graph from
the assign method
"""
# only functions that are actuall called are converted into ast
# func C is called twice and therefore shows up twice in the ast
FunctionCallVisitor().visit(tree)

c
b
c
a


In [7]:
"""
lets find an assign statement since assign statements kick off execution.

there are other ast types such as ast.Assign, ast.FunctionDef, ast.Import...
"""
for i, statement in enumerate(tree.body):
    if isinstance(statement, ast.Assign):  # https://stackoverflow.com/questions/37217823/how-to-find-detect-if-a-build-in-function-is-used-in-python-ast
        print(">> at index", i, ": \n", ast.dump(statement))
        idx = i

>> at index 5 : 
 Assign(targets=[Name(id='y', ctx=Store())], value=Call(func=Name(id='a', ctx=Load()), args=[], keywords=[]))


In [8]:
# now that we have the correct idx, we can grab the node corresponding to the assign statement
curr = tree.body[idx]  
ast.dump(curr)

"Assign(targets=[Name(id='y', ctx=Store())], value=Call(func=Name(id='a', ctx=Load()), args=[], keywords=[]))"

In [9]:
# find the functions that the assign statement calls
callable_name = find_callables(curr)
print(callable_name)

['a']


In [13]:
fetch("a", tree)

>> copying code for a ... 
def a():
    x = 2
    return c() + b() + x

>> copying code for b ... 
def b():
    return 2 + c()

>> copying code for c ... 
def c():
    return 3



In [12]:
def fetch(starting_node_name, tree):
    """
    starting_node_name should be the name of the callable in the assign statement.
    it is a str, eg "a"
    """
    stack = [starting_node_name]
    seen = []
    while len(stack) > 0:
        name = stack.pop()
        print(">> copying code for", name, "... ")

        # put the name in seen stack so we dont double count
        seen.append(name)  

        # find a's index number
        idx = name2idx(name)  

        # from index, retrieve correspoding node from the main tree and use it as curr
        curr = tree.body[idx]  

        # copy its source code
        print(astor.to_source(curr))  

        # find all callables for curr
        callable_name = find_callables(curr)  

        # and add it to the stack if we havent seen it yet
        for cn in callable_name:
            if (cn not in seen) and (cn not in stack):
                stack.append(cn)